In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split

df = pd.read_csv('../data/data_sample.csv')[:10]

label = list(df.columns)
label.remove('text')

X = df['text']
Y = df[label]

# Train test split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)


train_df


,text,SDG1,SDG2,SDG3,SDG4,SDG5,SDG6,SDG7,SDG8,SDG9,SDG10,SDG11,SDG12,SDG13,SDG14,SDG15,SDG16,SDG17
1,15 July 2019: An event on the sidelines of the...,0,0,0,0,0,0,0,1,0,1,0,0,1,1,1,0,0
5,24 September 2019: The Sustainable Development...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,The Pacific Economic Cooperation Council (PECC...,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,1
7,"The Ottawa Group, comprised of 13 World Trade ...",0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1
2,The 33rd UN-Water Meeting offered a brief chan...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
9,18 September 2019: UN Secretary-General Antóni...,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
4,14 October 2019: Building cities sustainable e...,1,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1
3,May 2019: Participants from multi-stakeholder ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,3 October 2019: Rainforest Foundation Norway h...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0


In [3]:
from transformers import BertTokenizer

BERT_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

/usr/local/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [4]:
from tokenizer import BERTTokenizerPooled

bert_pooled = BERTTokenizerPooled(tokenizer)
sample_text = train_df['text'][0]
sample_token = bert_pooled.preprocess(sample_text)
print(sample_token['input_ids'].shape)
# len(list(filter(lambda x: x != 0, sample_token['input_ids'][1])))

Token indices sequence length is longer than the specified maximum sequence length for this model (576 > 512). Running this sequence through the model will result in indexing errors


torch.Size([2, 512])


In [5]:
from module import SDGDataModule

data_module = SDGDataModule(train_df, test_df, tokenizer)
data_module.setup()

train_loader = data_module.train_dataloader()
train_dataset = data_module.train_dataset

first = train_dataset[0]
first['input_ids']

tensor([ 101, 1405, 1351,  ...,    0,    0,    0])

In [11]:
from pipeline import Pipeline
from model import SDGTagger

N_EPOCHS = 10
BATCH_SIZE = 12

steps_per_epoch=len(train_df) // BATCH_SIZE
total_training_steps = steps_per_epoch * N_EPOCHS
warmup_steps = total_training_steps // 5

model = SDGTagger(
    n_warmup_steps=warmup_steps,
    n_training_steps=total_training_steps 
)
ppln = Pipeline()
ppln.train(model, train_loader)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:147: LightningDeprecationWarning: Setting `Trainer(checkpoi

Epoch 0:   0%|          | 0/2 [00:00<?, ?it/s] 
torch.Size([6, 1536])
torch.Size([6, 17])


ValueError: not enough values to unpack (expected 2, got 0)

In [ ]:
"""SDG1
SDG2
SDG3
SDG4
SDG5
SDG6
SDG7
SDG8
SDG9
SDG10
SDG11
SDG12
SDG13
SDG14
SDG15
SDG16
SDG17""".split('\n')